# In this notebook we will perform hyperparameter tuning and training of adversarial anomaly detection model. 
#### For more details about this model refer to https://arxiv.org/pdf/1905.11034.pdf.

In [2]:
spark

The code failed because of a fatal error:
	Session 15 unexpectedly reached final status 'dead'. See logs:
stdout: 
2020-12-14 19:29:20,085 WARN  NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2020-12-14 19:29:20,193 WARN  DependencyUtils: Local jar /srv/hops/spark/jars/datanucleus-api.jar does not exist, skipping.
2020-12-14 19:29:20,350 INFO  RMProxy: Connecting to ResourceManager at resourcemanager.service.consul/10.0.0.4:8032
2020-12-14 19:29:21,267 INFO  Client: Requesting a new application from cluster with 1 NodeManagers
2020-12-14 19:29:21,345 INFO  Configuration: found resource resource-types.xml at file:/srv/hops/hadoop-3.2.0.1-EE-RC0/etc/hadoop/resource-types.xml
2020-12-14 19:29:21,354 INFO  ResourceUtils: Adding resource type - name = yarn.io/gpu, units = , type = COUNTABLE
2020-12-14 19:29:21,371 INFO  Client: Verifying our application has not requested more than the maximum memory capability of the c

## Connect to hsfs and retrieve datasets for training and evaluation 

In [3]:
import hsfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

ben_td = fs.get_training_dataset("gan_non_sar_training_df", 1)
eval_td = fs.get_training_dataset("gan_eval_df", 1)

The code failed because of a fatal error:
	Session 15 unexpectedly reached final status 'dead'. See logs:
stdout: 
2020-12-14 19:29:20,085 WARN  NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2020-12-14 19:29:20,193 WARN  DependencyUtils: Local jar /srv/hops/spark/jars/datanucleus-api.jar does not exist, skipping.
2020-12-14 19:29:20,350 INFO  RMProxy: Connecting to ResourceManager at resourcemanager.service.consul/10.0.0.4:8032
2020-12-14 19:29:21,267 INFO  Client: Requesting a new application from cluster with 1 NodeManagers
2020-12-14 19:29:21,345 INFO  Configuration: found resource resource-types.xml at file:/srv/hops/hadoop-3.2.0.1-EE-RC0/etc/hadoop/resource-types.xml
2020-12-14 19:29:21,354 INFO  ResourceUtils: Adding resource type - name = yarn.io/gpu, units = , type = COUNTABLE
2020-12-14 19:29:21,371 INFO  Client: Verifying our application has not requested more than the maximum memory capability of the c

In [4]:
from hops import experiment
from hops import hdfs
import json
emb_best_hyperparams_path = "Resources/embeddings_best_hp.json"
emb_best_hyperparams = json.loads(hdfs.load(emb_best_hyperparams_path))
emb_args_dict = {}
for key in emb_best_hyperparams.keys():
    emb_args_dict[key] = [emb_best_hyperparams[key]]

best_hyperparams_path = "Resources/gan_best_hp.json"
best_hyperparams = json.loads(hdfs.load(best_hyperparams_path))
args_dict = {}
for key in best_hyperparams.keys():
    args_dict[key] = [best_hyperparams[key]]
    

The code failed because of a fatal error:
	Session 15 unexpectedly reached final status 'dead'. See logs:
stdout: 
2020-12-14 19:29:20,085 WARN  NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2020-12-14 19:29:20,193 WARN  DependencyUtils: Local jar /srv/hops/spark/jars/datanucleus-api.jar does not exist, skipping.
2020-12-14 19:29:20,350 INFO  RMProxy: Connecting to ResourceManager at resourcemanager.service.consul/10.0.0.4:8032
2020-12-14 19:29:21,267 INFO  Client: Requesting a new application from cluster with 1 NodeManagers
2020-12-14 19:29:21,345 INFO  Configuration: found resource resource-types.xml at file:/srv/hops/hadoop-3.2.0.1-EE-RC0/etc/hadoop/resource-types.xml
2020-12-14 19:29:21,354 INFO  ResourceUtils: Adding resource type - name = yarn.io/gpu, units = , type = COUNTABLE
2020-12-14 19:29:21,371 INFO  Client: Verifying our application has not requested more than the maximum memory capability of the c

In [5]:
class ServingModule(tf.Module):
  def __init__(self, model):
    self.model = model

  @tf.function(input_signature=[tf.TensorSpec(shape=(None, 1), dtype=tf.float32)])
  def score(self, transaction):
    result = self.model(waveform)
    return { "scores": results }

The code failed because of a fatal error:
	Invalid status code '404' from http://10.0.0.4:8998/sessions/14 with error payload: {"msg":"Session '14' not found."}.

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


### Define hopsworks experiments wrapper function and put all the training logic there. 

In [4]:
def experiment_wrapper():
    
    latent_dim = args_dict['latent_dim'][0]
    discriminator_n_layers = args_dict['discriminator_n_layers'][0]
    discriminator_batch_norm = args_dict['discriminator_batch_norm'][0]
    discriminator_dropout_rate = args_dict['discriminator_dropout_rate'][0]
    discriminator_learning_rate = args_dict['discriminator_learning_rate'][0]
    discriminator_extra_steps = args_dict['discriminator_extra_steps'][0]

    generator_start_n_units = args_dict['generator_start_n_units'][0]
    generator_n_layers = args_dict['generator_n_layers'][0]
    generator_activation_fn = args_dict['generator_activation_fn'][0]
    generator_batch_norm = args_dict['generator_batch_norm'][0]
    generator_dropout_rate = args_dict['generator_dropout_rate'][0]
    generator_learning_rate = args_dict['generator_learning_rate'][0]

    encoder_start_n_units = args_dict['encoder_start_n_units'][0]
    encoder_n_layers = args_dict['encoder_n_layers'][0]
    encoder_activation_fn = args_dict['encoder_activation_fn'][0]
    encoder_batch_norm = args_dict['encoder_batch_norm'][0]
    encoder_dropout_rate = args_dict['encoder_dropout_rate'][0]
    encoder_learning_rate  = args_dict['encoder_learning_rate'][0]

    import os
    import sys
    import uuid
    import random    
    
    import tensorflow as tf
    from adversarialaml.gan_enc_ano import GanAnomalyDetector,  GanAnomalyMonitor 
    from hops import tensorboard
    from hops import model as hops_model

    # Set the number of epochs for trainining.
    EPOCHS = 2

    train_input = ben_td.tf_data(target_name='target', is_training=True)
    train_input_processed = train_input.tf_record_dataset(process=True, batch_size=16, num_epochs=EPOCHS)
    eval_input = ben_td.tf_data(target_name='target', is_training=True)
    eval_input_processed = eval_input.tf_record_dataset(process=True, batch_size=1, num_epochs=EPOCHS)    
    
    if discriminator_dropout_rate > 0.0:
        discriminator_batch_dropout = True
    else:
        discriminator_batch_dropout = False
    

    if discriminator_dropout_rate > 0.0:
        generator_batch_dropout=True
    else:
        generator_batch_dropout=False

    if encoder_dropout_rate > 0.0:
        encoder_batch_dropout=True
    else:
        encoder_batch_dropout=False   


    if discriminator_batch_norm==0:
        discriminator_batch_norm = False
    else:
        discriminator_batch_norm = True
        
    if generator_activation_fn==0:
        generator_activation_fn="tanh"
    else:    
        generator_activation_fn="relu"
        
    if generator_batch_norm==0:
        generator_batch_norm = False
    else:
        generator_batch_norm = True
        
    if encoder_activation_fn==0:
        encoder_activation_fn="tanh"
    else:    
        encoder_activation_fn="relu"
        
    if encoder_batch_norm==0:
        encoder_batch_norm=False
    else:
        encoder_batch_norm=True        
        

    discriminator_double_neurons=False
    discriminator_bottleneck_neurons=True
    generator_double_neurons=True
    generator_bottleneck_neurons=False
        
    # Instantiate the GanAnomalyDetector model.
    gan_anomaly_detector = GanAnomalyDetector(
                input_dim=emb_args_dict['emb_size'][0],
                latent_dim=latent_dim,

                discriminator_start_n_units=emb_args_dict['emb_size'][0],
                discriminator_n_layers=discriminator_n_layers,
                discriminator_activation_fn="sigmoid",
                discriminator_double_neurons=discriminator_double_neurons,
                discriminator_bottleneck_neurons=discriminator_bottleneck_neurons,
                discriminator_batch_norm=discriminator_batch_norm,
                discriminator_batch_dropout=discriminator_batch_dropout,
                discriminator_dropout_rate=discriminator_dropout_rate,
                discriminator_learning_rate=discriminator_learning_rate,
                discriminator_extra_steps=discriminator_extra_steps,

                generator_start_n_units=generator_start_n_units,
                generator_n_layers=generator_n_layers,
                generator_activation_fn=generator_activation_fn,
                generator_double_neurons=generator_double_neurons,
                generator_bottleneck_neurons=generator_bottleneck_neurons,
                generator_batch_norm=generator_batch_norm,
                generator_batch_dropout=generator_batch_dropout,
                generator_dropout_rate=generator_dropout_rate,
                generator_learning_rate=generator_learning_rate,

                encoder_start_n_units=encoder_start_n_units,
                encoder_n_layers=encoder_n_layers,
                encoder_activation_fn=encoder_activation_fn,
                encoder_batch_norm=encoder_batch_norm,
                encoder_batch_dropout=encoder_batch_dropout,
                encoder_dropout_rate=encoder_dropout_rate,
                encoder_learning_rate=encoder_learning_rate,

    )
        
    # Compile the WGAN model.
    gan_anomaly_detector.compile()

    callbacks = [
        #GanAnomalyMonitor(batch_size=1, latent_dim=4, input_dim=16, alpha=0.7, real_data=eval_input_processed),
        tf.keras.callbacks.TensorBoard(log_dir=tensorboard.logdir()),
        tf.keras.callbacks.ModelCheckpoint(filepath=tensorboard.logdir()),
    ]
    # Start training the model.
    history = gan_anomaly_detector.fit(train_input_processed, callbacks=[callbacks])

    metrics={'loss': history.history["g_loss"][0]} 

    # save to the model registry
    export_path = os.getcwd() + '/model-' + str(uuid.uuid4())
    print('Exporting trained model to: {}'.format(export_path))
    tf.saved_model.save(gan_anomaly_detector, export_path, signatures={ "score": module.outputs})
    print('Done exporting!')
        
    hops_model.export(export_path, 'ganAml', metrics=metrics)
    
    return metrics

## Use above experiments wrapper function to conduct hops training experiments.

In [5]:
from hops import experiment
    
experiment.launch(experiment_wrapper,  name='train_gan_aml', metric_key='loss', local_logdir=False)

Finished Experiment 

('hdfs://rpc.namenode.service.consul:8020/Projects/amlsim/Experiments/application_1607949680860_0012_1', {'loss': -0.6464197635650635, 'log': 'Experiments/application_1607949680860_0012_1/output.log'})